In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras import regularizers

from  IPython import display
from matplotlib import pyplot as plt

import numpy as np

import pathlib
import shutil
import tempfile
import tensorflow.compat.v1 as tf1

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
device_name = tf.test.gpu_device_name()
if "GPU" not in device_name:
    print("GPU device not found")
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
directory = '/kaggle/input/navires-2023-libre'  # Chemin vers le dossier contenant vos 10 dossiers de catégories
data_dir = directory + '/ships16x24/ships_16x24_10cat/data'
test_file = directory + '/test.npy'

In [5]:
image_width, image_height = 16,24  # Dimensions de vos images
batch_size = 32  # Taille du lot d'images à générer
num_classes = 10  # Nombre total de catégories
class_weights = [1.15,0.6,1.25,0.6,1.8,0.6,1.1,1.18,1.30,1.15]
class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}

In [6]:
strategy = tf.distribute.MirroredStrategy()
print("--> Checking for physical Tensorflow devices")
for device in tf.config.list_physical_devices():
    print(": {}".format(device.name))

--> Checking for physical Tensorflow devices
: /physical_device:CPU:0
: /physical_device:GPU:0


In [7]:

#with strategy.scope():
with tf.device('/gpu:0'):
    # instantiate your model here
    model =  models.Sequential([
                    layers.Resizing(24,24,interpolation='bicubic', input_shape=(image_width, image_height, 3)),
                    layers.RandomZoom(height_factor=(-0.1, 0.1),width_factor=(-0.1, 0.1)),
                    layers.RandomFlip('horizontal'),
                    layers.Rescaling(1. / 255),
                    layers.Conv2D(256, (3, 3), activation='relu'),
                    layers.BatchNormalization(),
                    layers.Conv2D(256, (3, 3), activation='relu'),
                    layers.BatchNormalization(),
                    layers.MaxPooling2D((2, 2)),
                    layers.Dropout(0.2),
                    layers.Conv2DTranspose(128, (3, 3), activation='relu', padding='same'),
                    #layers.BatchNormalization(),
                    layers.Conv2DTranspose(128, (3, 3), activation='relu', padding='same'),
                    layers.BatchNormalization(),
                    layers.MaxPooling2D((2, 2)),
                    layers.Dropout(0.2),
                    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
                    layers.BatchNormalization(),
                    layers.MaxPooling2D((2, 2)),
                    layers.Dropout(0.2),
                    layers.Flatten(),                
                    layers.Dense(512, activation="relu"),
                    layers.BatchNormalization(),
                    layers.Dropout(0.1),
                    layers.Dense(256, activation="relu"),
                    layers.BatchNormalization(),
                    layers.Dropout(0.7),
                    layers.Dense(10, activation='softmax')])




In [8]:
from keras.optimizers import Adadelta, Adam, Adamax




In [9]:
model.compile(optimizer=Adamax(0.005), loss='categorical_crossentropy', metrics=['accuracy'])


In [10]:
model.build()
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing (Resizing)         (None, 24, 24, 3)         0         
                                                                 
 random_zoom (RandomZoom)    (None, 24, 24, 3)         0         
                                                                 
 random_flip (RandomFlip)    (None, 24, 24, 3)         0         
                                                                 
 rescaling (Rescaling)       (None, 24, 24, 3)         0         
                                                                 
 conv2d (Conv2D)             (None, 22, 22, 256)       7168      
                                                                 
 batch_normalization (BatchN  (None, 22, 22, 256)      1024      
 ormalization)                                                   
                                                        

In [11]:
print(len(model.layers))

27


In [12]:
#data
seed = 100
np.random.seed(seed)
tf.random.set_seed(seed)
train = keras.utils.image_dataset_from_directory(data_dir,seed=seed, labels='inferred', label_mode='categorical', image_size=(image_width,image_height), validation_split=0.2, subset="training")
validation = keras.utils.image_dataset_from_directory(data_dir,seed=seed, labels='inferred', label_mode='categorical', image_size=(image_width,image_height), validation_split=0.2, subset="validation")

Found 35515 files belonging to 10 classes.
Using 28412 files for training.
Found 35515 files belonging to 10 classes.
Using 7103 files for validation.


In [13]:
print(train.class_names, train, validation)

['coastguard_scaled', 'containership_scaled', 'corvette_scaled', 'cruiser_scaled', 'cv_scaled', 'destroyer_scaled', 'methanier_scaled', 'smallfish_scaled', 'submarine_scaled', 'tug_scaled'] <_BatchDataset element_spec=(TensorSpec(shape=(None, 16, 24, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))> <_BatchDataset element_spec=(TensorSpec(shape=(None, 16, 24, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))>


In [14]:
with tf.device('/gpu:0'):
    model.fit(train, validation_data=(validation), batch_size=256, epochs=50)

Epoch 1/50


2023-05-26 13:55:34.599926: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


888/888 [==============================] - 61s 60ms/step - loss: 2.3577 - accuracy: 0.2585 - val_loss: 1.8854 - val_accuracy: 0.3261
Epoch 2/50
888/888 [==============================] - 17s 19ms/step - loss: 1.6969 - accuracy: 0.4167 - val_loss: 1.5860 - val_accuracy: 0.4377
Epoch 3/50
888/888 [==============================] - 17s 19ms/step - loss: 1.4444 - accuracy: 0.5013 - val_loss: 1.3839 - val_accuracy: 0.5160
Epoch 4/50
888/888 [==============================] - 17s 19ms/step - loss: 1.3274 - accuracy: 0.5449 - val_loss: 1.3716 - val_accuracy: 0.5222
Epoch 5/50
888/888 [==============================] - 17s 20ms/step - loss: 1.2412 - accuracy: 0.5747 - val_loss: 1.3781 - val_accuracy: 0.5243
Epoch 6/50
888/888 [==============================] - 17s 19ms/step - loss: 1.1596 - accuracy: 0.6064 - val_loss: 1.2752 - val_accuracy: 0.5630
Epoch 7/50
888/888 [==============================] - 18s 20ms/step - loss: 1.1022 - accuracy: 0.6266 - val_loss: 1.1068 - val_accuracy: 0.6199
Epo

In [15]:

# Sauvegardez le modèle
model.save('/kaggle/working/mon_modele3.h5')

In [16]:
#model = models.load_model('/kaggle/working/mon_modele.h5')

In [17]:
test_data = np.load(test_file)
predictions = model.predict(test_data)

116/116 [==============================] - 1s 5ms/step


In [18]:
predicted_labels = np.argmax(predictions, axis=1)
print(len(predicted_labels))

3698


In [19]:
print("Nombre de couches : ", len(model.layers))

Nombre de couches :  27


In [20]:
df = pd.DataFrame({"Category":predicted_labels})
df.to_csv("submission.csv", index_label="Id")

import os
os.chdir(r'/kaggle/working')
from IPython.display import FileLink
FileLink(r'submission.csv')

/kaggle/working/submission.csv

In [21]:
!head reco_nav.csv

head: cannot open 'reco_nav.csv' for reading: No such file or directory
